In [1]:
from openai import OpenAI
import pandas as pd
from datetime import datetime
from typing import Optional


In [2]:
def df_to_str(df: pd.DataFrame, sort_by: Optional[str] = None, ascending: Optional[bool] = 1) -> str:
    if sort_by:
        df=df.sort_values(sort_by,ascending=1)
    df_string = ""
    for index, row in df.iterrows():
        for col in df.columns:
            df_string += f"{col}: {row[col]}\n"
        df_string += "\n"  # Add a newline for separation between records if there are multiple rows
    return df_string

def retrieve_context_chatbot(ml_id):
    relevant_ml_cases_info=ml_cases[ml_cases['ML_ID']==ml_id][ml_cases_cols]
    relevant_final_rel=final_relationship[final_relationship['ML_ID']==ml_id][final_relationship_cols]
    ml_type=ml_cases[ml_cases['ML_ID']==ml_id]['ML_type'].iloc[0]
    context = f'''*{ml_id}*
Terduga kasus money laundering jenis: {ml_type}
Informasi transaksi dan keterkaitan:
{df_to_str(relevant_final_rel,'TIMESTAMP')}

Rangkuman:
{df_to_str(relevant_ml_cases_info)}'''
    return context

In [31]:
#load context from table

ml_cases_cols=['n_entities_involved','total_amount','ML_type']
final_relationship_cols = ['RECEIVER_ACCOUNT', 'SENDER_ACCOUNT', 'TX_TYPE', 'TX_AMOUNT', 'TIMESTAMP']

# ml_case and final_relationship as global variables
ml_cases=pd.read_csv('ml_cases_finale2.csv')
final_relationship=pd.read_csv('final_relationship_finale.csv')

#initiate openai api here
key_api=#censored for noe
oai_client=OpenAI(api_key=key_api)

In [43]:
# Load your data
# ml_cases = pd.read_csv('ml_cases_finale2.csv')
# final_relationship = pd.read_csv('final_relationship_v_finale.csv')

ml_cases_cols = ['n_entities_involved', 'total_amount', 'ML_type']
final_relationship_cols = ['RECEIVER_ACCOUNT', 'SENDER_ACCOUNT', 'TX_TYPE', 'TX_AMOUNT', 'TIMESTAMP']

key_api = env.api_ai
oai_client = OpenAI(api_key=key_api)

class History:
    def __init__(self):
        self.histories = {}

    def create_chat(self, chat_id):
        if chat_id not in self.histories:
            self.histories[chat_id] = pd.DataFrame(columns=['timestamp', 'sender', 'message'])

    def append_to_chat_history(self, chat_id, sender, message):
        if chat_id not in self.histories:
            self.create_chat(chat_id)
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_entry = pd.DataFrame([[timestamp, sender, message]], columns=['timestamp', 'sender', 'message'])
        self.histories[chat_id] = pd.concat([self.histories[chat_id], new_entry], ignore_index=True)

    def get_history(self, chat_id):
        if chat_id in self.histories:
            return self.histories[chat_id]
        else:
            return pd.DataFrame(columns=['timestamp', 'sender', 'message'])

    def save_to_csv(self, chat_id, filename):
        if chat_id in self.histories:
            self.histories[chat_id].to_csv(filename, index=False)

    def load_from_csv(self, chat_id, filename):
        self.histories[chat_id] = pd.read_csv(filename)

    def history_to_string(self, chat_id):
        if chat_id in self.histories:
            df = self.histories[chat_id]
            history_string = ""
            for index, row in df.iterrows():
                history_string += f"{row['timestamp']} - {row['sender']}: {row['message']}\n"
            return history_string
        else:
            return ""

class Chatbot:
    def __init__(self, ml_id, user_id):
        self.history = History()
        self.ml_id = ml_id
        self.system_prompt=''
        self.chat_id =f"{user_id}"
        # f"{user_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    def chat_with_model(self, prompt, model='gpt-4o'):
        chat_history_str = self.history.history_to_string(self.chat_id)
        if chat_history_str: 
            chat_history_context = f'''Berikut perbincangan pada pesan sebelumnya: {chat_history_str}'''
        else:
            chat_history_context = ''

        system_prompt = f'''Kamu adalah asisten analis fraud dari Drovide Pivide. 
        Kamu menanggapi pertanyaan secara singkat, padat, dan jelas berdasarkan materi: 
        {retrieve_context_chatbot(self.ml_id)} dan pengetahuanmu tentang fraud di institusi finansial.
        untuk tipe transaksi Direktur dan Komisaris maka penerimanya adalah perusahaan dan pengirimnya adalah stakeholder sesuai yang tertulis pada tipe transaksi. 
        Jangan dianggap sebagai tipe transaksi tapi sebagai hubungan stakeholder.
        {chat_history_context}
        '''

        completion = oai_client.chat.completions.create(
            model=model,
            temperature=0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content.strip()
    # system_prompt 
    

    def add_message(self, sender, message):
        self.history.append_to_chat_history(self.chat_id, sender, message)

    # def start_conversation(self, user_id):
    #     chat_id = f"{user_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    #     while True:
    #         user_input = input("You: ")
    #         if user_input.lower() == 'exit':
    #             break
    #         assistant_response = self.chat_with_model(user_input, chat_id)
    #         self.add_message(chat_id, 'user', user_input)
    #         self.add_message(chat_id, 'assistant', assistant_response)

    #     print(self.history.get_history(chat_id))
    #     self.history.save_to_csv(chat_id, f'chat_history_{chat_id}.csv')

def df_to_str(df: pd.DataFrame, sort_by: Optional[str] = None, ascending: Optional[bool] = 1) -> str:
    if sort_by:
        df = df.sort_values(sort_by, ascending=ascending)
    df_string = ""
    for index, row in df.iterrows():
        for col in df.columns:
            df_string += f"{col}: {row[col]}\n"
        df_string += "\n"
    return df_string

def retrieve_context_chatbot(ml_id):
    relevant_ml_cases_info = ml_cases[ml_cases['ML_ID'] == ml_id][ml_cases_cols]
    relevant_final_rel = final_relationship[final_relationship['ML_ID'] == ml_id][final_relationship_cols]
    ml_type = ml_cases[ml_cases['ML_ID'] == ml_id]['ML_type'].iloc[0]
    context = f'''*{ml_id}*
Terduga kasus money laundering jenis: {ml_type}
Informasi transaksi dan keterkaitan:
{df_to_str(relevant_final_rel, 'TIMESTAMP')}

Rangkuman:
{df_to_str(relevant_ml_cases_info)}'''
    # print (df_to_str(relevant_final_rel, 'TIMESTAMP'))
    return context


def chat(prompt,chatbot):
    # data = request.json
    # user_id = data['user_id']
    # ml_id = data['ml_id']
    # prompt = data['prompt']
    
    # chat_id = f"{user_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    chatbot.add_message( 'user', prompt)
    assistant_response = chatbot.chat_with_model(prompt)
    chatbot.add_message( 'assistant', assistant_response)
    
    print(prompt)
    print(assistant_response)

   

In [44]:
user_id = 'enjang69'  
ml_id='cycle_1'
chatbot = Chatbot(ml_id, user_id)

In [45]:
prompt='halo, jelasin singkat'

chat(prompt,chatbot)

halo, jelasin singkat
Kasus ini melibatkan 8 entitas dengan total transaksi sebesar 15 juta. Jenis money laundering yang terdeteksi adalah "cycle". Transaksi mencurigakan melibatkan perpindahan dana melalui beberapa akun, termasuk hubungan stakeholder antara Musmuliadi dan perusahaan Masa Metonia Abadi.


In [46]:
prompt='siapa entitas terlibat'

chat(prompt,chatbot)

siapa entitas terlibat
Entitas yang terlibat adalah:
1. Masa Metonia Abadi
2. Musmuliadi
3. Asep Ramadhan
4. Budi Prasetyo
5. Budi Setiawan
6. Dewi Sartika
7. Citra Lestari
8. Ayu Lestari


In [47]:
prompt='yang erusahaan'

chat(prompt,chatbot)

yang erusahaan
Entitas perusahaan yang terlibat adalah Masa Metonia Abadi.


In [48]:
prompt='mengirim ke siapa dia?'

chat(prompt,chatbot)

mengirim ke siapa dia?
Masa Metonia Abadi mengirim dana ke Citra Lestari.


In [49]:
prompt='apakah ada penerimanya'

chat(prompt,chatbot)

apakah ada penerimanya
Ya, penerima dari transaksi yang dilakukan oleh Masa Metonia Abadi adalah Citra Lestari.


In [50]:
prompt='apakah dia menerima?'

chat(prompt,chatbot)

apakah dia menerima?
Tidak, Masa Metonia Abadi tidak menerima dana dalam transaksi yang tercatat. Masa Metonia Abadi hanya mengirim dana ke Citra Lestari.


In [52]:
prompt='lalu siapa yang mennghubungkan perusahaan ini selain orang tadi'

chat(prompt,chatbot)

lalu siapa yang mennghubungkan perusahaan ini selain orang tadi
Selain Citra Lestari, yang menghubungkan perusahaan Masa Metonia Abadi adalah Musmuliadi. Musmuliadi memiliki hubungan stakeholder dengan perusahaan tersebut.


In [42]:
prompt='apakah ada penerimanya dari entitas sblmnya'

chat(prompt,chatbot)

apakah ada penerimanya dari entitas sblmnya
Tidak, tidak ada transaksi yang melibatkan akun dengan nama "enjang69" sebagai penerima.


In [30]:
final_relationship

,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,ML_ID,TX_TYPE
0,Bang Windah Bersaudara,Aliya Zahra,IDR,0.0,8.528834e+07,2022-08-01 11:40:00,gather-scatter_1,wire
1,Bang Windah Bersaudara,Ratna Sari,IDR,3.0,9.974407e+07,2022-08-05 13:16:00,gather-scatter_1,wire
2,Bang Windah Bersaudara,Dinda Puspita,IDR,718.0,8.912334e+07,2022-08-07 09:23:00,gather-scatter_1,wire
3,Bang Windah Bersaudara,Sulastri,IDR,11.0,9.654125e+07,2022-08-08 11:24:00,gather-scatter_1,wire
4,Bang Windah Bersaudara,Dewi Lestari,IDR,0.0,8.163624e+07,2022-08-16 14:39:00,gather-scatter_1,wire
...,...,...,...,...,...,...,...,...
210,Tatang Supardi,Asep Wahyu,RM,2035.0,2.900000e+04,2022-08-27 15:04:00,fan-out_5,wire
211,Ujang Suryana,Asep Wahyu,IDR,13514.0,9.565582e+07,2022-08-28 10:06:00,fan-out_5,wire
212,Vivi Setiawati,Asep Wahyu,USD,43948.0,5.932057e+04,2022-08-28 17:23:00,fan-out_5,wire
213,Wawan Sopian,Asep Wahyu,RM,11469.0,2.900000e+04,2022-09-04 09:39:00,fan-out_5,wire


In [18]:
prompt='siapa aja orang terlibt'

chat(prompt,chatbot)

siapa aja orang terlibt
Kasus ini melibatkan 8 entitas. Namun, dari informasi yang diberikan, hanya satu nama yang disebutkan, yaitu "enjang69". Untuk mengetahui entitas lainnya, diperlukan informasi tambahan.


In [6]:
class History:
    def __init__(self):
        self.histories = {}

    def create_chat(self, chat_id):
        if chat_id not in self.histories:
            self.histories[chat_id] = pd.DataFrame(columns=['timestamp', 'sender', 'message'])

    def append_to_chat_history(self, chat_id, sender, message):
        if chat_id not in self.histories:
            self.create_chat(chat_id)
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_entry = pd.DataFrame([[timestamp, sender, message]], columns=['timestamp', 'sender', 'message'])
        self.histories[chat_id] = pd.concat([self.histories[chat_id], new_entry], ignore_index=True)

    def get_history(self, chat_id):
        if chat_id in self.histories:
            return self.histories[chat_id]
        else:
            return pd.DataFrame(columns=['timestamp', 'sender', 'message'])

    def save_to_csv(self, chat_id, filename):
        if chat_id in self.histories:
            self.histories[chat_id].to_csv(filename, index=False)

    def load_from_csv(self, chat_id, filename):
        self.histories[chat_id] = pd.read_csv(filename)

    def history_to_string(self, chat_id):
        if chat_id in self.histories:
            df = self.histories[chat_id]
            history_string = ""
            for index, row in df.iterrows():
                history_string += f"{row['timestamp']} - {row['sender']}: {row['message']}\n"
            return history_string
        else:
            return ""
        
class Chatbot:
    def __init__(self, ml_id):
        self.history = History()
        
    def chat_with_model(self, prompt,chat_id, model='gpt-4o'):
        chat_history_str = self.history.history_to_string(chat_id)
        if chat_history_str: 
            chat_history_context= f'''Berikut perbincangan pada pesan sebelumnya: {chat_history_str}'''
            # print(f'chat_history_str presents',chat_history_str)
        else:
            chat_history_context=''
            # print(f'chat_history_str non existent')
        system_prompt=f'''Kamu adalah asisten analis fraud dari Drovide Pivide. 
        Kamu menanggapi pertanyaan secara singkat, padat, dan jelas berdasarkan materi: 
        {retrieve_context_chatbot(ml_id)} dan pengetahuanmu tentang fraud di institusi finansial.
        {chat_history_context}
        '''
        
        completion = oai_client.chat.completions.create(
            model=model,
            temperature=0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        )
        return chat_history_str #system_prompt #completion.choices[0].message.content.strip()

    def add_message(self, chat_id, sender, message):
        self.history.append_to_chat_history(chat_id, sender, message)

    def start_conversation(self, user_id):
        # Generate a unique chat_id based on user_id and current timestamp
        chat_id = f"{user_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        # self.add_message(chat_id, 'system', self.system_prompt)

        while True:
            user_input = input("You: ")
            if user_input.lower() == 'exit':
                break
            print(f"User: {user_input}")

            assistant_response = self.chat_with_model(user_input, chat_id)
           
            print(f"Assistant: {assistant_response}",'\n')
            
            self.add_message(chat_id, 'user', user_input)
            self.add_message(chat_id, 'assistant', assistant_response)

        # Print the chat history at the end
        print(self.history.get_history(chat_id))

        # Optionally, save the chat history
        self.history.save_to_csv(chat_id, f'chat_history_{chat_id}.csv')

# Example usage
user_id = 'enjang69'  

ml_id='20240524-roundtrip-01'
chatbot = Chatbot(ml_id)
chatbot.start_conversation(user_id)

User: 1
Assistant:  

User: 2
Assistant: 2024-06-05 15:16:51 - user: 1
2024-06-05 15:16:51 - assistant: 
 

User: 3
Assistant: 2024-06-05 15:16:51 - user: 1
2024-06-05 15:16:51 - assistant: 
2024-06-05 15:16:54 - user: 2
2024-06-05 15:16:54 - assistant: 2024-06-05 15:16:51 - user: 1
2024-06-05 15:16:51 - assistant: 

 

             timestamp     sender  \
0  2024-06-05 15:16:51       user   
1  2024-06-05 15:16:51  assistant   
2  2024-06-05 15:16:54       user   
3  2024-06-05 15:16:54  assistant   
4  2024-06-05 15:16:58       user   
5  2024-06-05 15:16:58  assistant   

                                             message  
0                                                  1  
1                                                     
2                                                  2  
3  2024-06-05 15:16:51 - user: 1\n2024-06-05 15:1...  
4                                                  3  
5  2024-06-05 15:16:51 - user: 1\n2024-06-05 15:1...  


In [9]:

chat_history = pd.DataFrame(columns=['timestamp', 'sender', 'message'])

C:\Users\BNI\AppData\Local\Temp\ipykernel_6984\758449357.py:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ml_cases = final_relationship.groupby("ML_ID").apply(lambda group: pd.Series({
